In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import numpy
import os
import tensorflow as tf

In [ ]:
train_set = numpy.load('train_data.npz')
x_train = train_set['arr_0']
y_train = train_set['arr_1']

In [ ]:
train_set = numpy.load('test_data.npz')
x_test = train_set['arr_0']
y_test = train_set['arr_1']

In [ ]:
val_set = numpy.load('validation_data.npz')
x_val = val_set['arr_0']
y_val = val_set['arr_1']

In [ ]:
x_train = numpy.expand_dims(x_train, axis = -1)
x_val = numpy.expand_dims(x_val, axis = -1)
x_test = numpy.expand_dims(x_test, axis = -1)

In [ ]:
# print(x_val.shape)

In [ ]:
def get_dataset(x_train, y_train, x_val, y_val):
  # x_train = tf.convert_to_tensor(x_train, dtype=x_train.dtype, dtype_hint=None, name='x_train').batch(batch_size)
  # y_train = tf.convert_to_tensor(y_train, dtype=y_train.dtype, dtype_hint=None, name='y_train').batch(batch_size)
  # x_val = tf.convert_to_tensor(x_val, dtype=x_val.dtype, dtype_hint=None, name='x_val').batch(batch_size)
  # y_val = tf.convert_to_tensor(y_val, dtype=y_val.dtype, dtype_hint=None, name='y_val').batch(batch_size)
  # train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(5000).batch(batch_size)
  # val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(batch_size)
  train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(5000)
  val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
  return train_dataset, val_dataset

In [ ]:
def init_model(input_shape):  
  n_classes = 278
  conv_kernel_size = (3, 1)
  lstm_hidden_units = 64
  # Input layer
  input_layer = tf.keras.layers.Input(input_shape, name='input')
  # Convolutional branch
  # convolution 1
  conv_1 = tf.keras.layers.Conv2D(16, conv_kernel_size, activation='relu')(input_layer)
  max_pool_1 = tf.keras.layers.MaxPooling2D(2, 2)(conv_1)
  # convolution 2
  conv_2 = tf.keras.layers.Conv2D(32, conv_kernel_size, activation='relu')(max_pool_1)
  max_pool_2 = tf.keras.layers.MaxPooling2D(2,2)(conv_2)
  # dropout_1 = tf.keras.layers.Dropout(0.5)(max_pool_2)
  # convolution 3
  conv_3 = tf.keras.layers.Conv2D(64, conv_kernel_size, activation='relu')(max_pool_2)
  max_pool_3 = tf.keras.layers.MaxPooling2D(2,2)(conv_3)
  # convolution 4
  conv_4 = tf.keras.layers.Conv2D(64, conv_kernel_size, activation='relu')(max_pool_3)
  max_pool_4 = tf.keras.layers.MaxPooling2D(4,4)(conv_4)
  # dropout_2 = tf.keras.layers.Dropout(0.5)(max_pool_4)
  conv_5 = tf.keras.layers.Conv2D(64, conv_kernel_size, activation='relu')(max_pool_4)
  max_pool_5 = tf.keras.layers.MaxPooling2D((4, 4))(conv_5)
  # Flatten
  flatten_1 = tf.keras.layers.Flatten()(max_pool_5)
  # Recurrent branch
  max_pool_lstm = tf.keras.layers.MaxPooling2D(2, 2)(input_layer)
  squeeze_lambda = tf.keras.layers.Lambda(lambda x: tf.keras.backend.squeeze(x, axis= -1))(max_pool_lstm)
  lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_hidden_units))(squeeze_lambda)
  # Concatenate convolutional and recurrent branches
  combine_cnn_rnn = tf.keras.layers.concatenate([flatten_1, lstm], axis=-1, name ='combine')
  # dropout_3 = tf.keras.layers.Dropout(0.5)(combine_cnn_rnn)
  output_layer = tf.keras.layers.Dense(n_classes, activation='softmax')(combine_cnn_rnn)
  model = tf.keras.models.Model(input_layer, output_layer)
  
  print(model.summary())
  return model

In [ ]:
# def train_model(x_train, y_train, x_val, y_val):
n_frequency = 128
n_frames = 862 

input_shape = (n_frames, n_frequency, 1)
reducelr_callback = tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_accuracy', factor=0.5, patience=10, min_delta=0.01,
            verbose=1
        )

callbacks_list = [reducelr_callback]


# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
# tf.config.experimental_connect_to_host(resolver.master())
# tf.tpu.experimental.initialize_tpu_system(resolver)
# strategy = tf.distribute.experimental.TPUStrategy(resolver)
# with strategy.scope():
model = init_model(input_shape)
opt = tf.keras.optimizers.RMSprop(lr=0.0005)  # Optimizer
model.compile(
        loss='categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy']
    )
# train_dataset, val_dataset = get_dataset(x_train, y_train, x_val, y_val)

In [ ]:
history = model.fit(x_train, y_train, epochs=50, batch_size=64, validation_data=(x_val, y_val), verbose=2, callbacks=callbacks_list)

In [ ]:
results = model.evaluate(x_test, y_test, batch_size=128)
print('test loss, test acc:', results)